In [22]:
# !pip install dash_bootstrap_components
# !pip install dash_daq
# !pip install dash_mantine_components

# EJjemplos
# https://github.com/lomska/Visualizing-Global-Trade-Networks
# https://github.com/plotly/dash-sample-apps/tree/main/apps/dash-manufacture-spc-dashboard

In [23]:
#####################################################
# Part 1: Import needed packages
#####################################################
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq
import dash_mantine_components as dmc

In [24]:
#####################################################
# Part 2: Basic app information
#####################################################
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Stock Analysis Dashboard"
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",}

In [25]:
from helpers_db import run_sql, get_engine
from helpers_queries import queries

engine = get_engine()
with engine.begin() as conn:
  balance = run_sql(conn, queries['balance']).mappings().first()
  df_positions = pd.DataFrame.from_dict(run_sql(conn, queries['positions_status']).mappings().all())
  df_crossing_levels = pd.DataFrame.from_dict(run_sql(conn, queries['crossing_levels']).mappings().all())
df_crossing_levels

,ago,crossing,direction,dist,dt,level,movement,symbol
0,"00 days ago, 01 hours ago",annual,bullish,0.000119,2024-03-28,0.10000000000000000000,0.000119,PYPL
1,"00 days ago, 01 hours ago",longterm,bullish,0.002180,2024-03-28,0.812,0.002185,WDAY
2,"00 days ago, 01 hours ago",longterm,bearish,0.005496,2024-03-28,1,-0.005466,DBK
3,"00 days ago, 01 hours ago",annual,bearish,0.030454,2024-03-28,0.50000000000000000000,-0.029554,GCT
4,"00 days ago, 01 hours ago",annual,bearish,0.012033,2024-03-28,0.40000000000000000000,-0.011890,META
5,"00 days ago, 01 hours ago",annual,bearish,0.001654,2024-03-28,-0.10000000000000000000,-0.001651,ON


In [26]:
#####################################################
# Part 4: App layout
#####################################################
color_negative      = '#FF5E5E'
color_positive      = '#B0C5A4'
classNameSixColumns = 'columns'
div_dropdown_symbol = html.Div(dcc.Dropdown(
    id='symbol-dropdown', value=df_positions.symbol.to_list(), clearable=False, multi=True,
    options=[{'label': x, 'value': x} for x in sorted(df_positions.symbol.unique())]
), className=classNameSixColumns, style={"width": "50%"}, )

div_dropdown_field = html.Div(dcc.Dropdown(
    id='field-dropdown', value='symbol', clearable=False,
    options=[{'label': x, 'value': x} for x in [i for j, i in enumerate(df_positions.columns) if j not in ['symbol']]]
), className=classNameSixColumns, style={"width": "25%"}, )

div_led_gains_today = html.Div(daq.LEDDisplay(
  id="led-gains-today",
  value='{:.3f}'.format(balance["profits_hist_perc"] * 100),
  color= color_positive if balance["profits_hist_perc"] > 0 else color_negative,
  size=50,
), className=classNameSixColumns, style={"width": "25%"}, )
div_led_gains_1d_ago = html.Div(daq.LEDDisplay(
  id="led-gains-1d_ago",
  value='{:.3f}'.format(balance["profits_1d_ago_perc"] * 100),
  color= color_positive if balance["profits_1d_ago_perc"] > 0 else color_negative,
  size=50,
), className='six columns', style={"width": "25%"}, )
div_led_gains_value = html.Div(daq.LEDDisplay(
  id="led-gains-history",
  value='{:.2f}'.format(balance["profit_hist_value"]),
  color= color_positive if balance["profit_hist_value"] > 0 else color_negative,
  size=50,
), className=classNameSixColumns, style={"width": "50%"}, )

app.layout = html.Div([
    html.Div([div_led_gains_1d_ago, div_led_gains_today, div_led_gains_value], className='row'),
    html.Div([html.Div([div_dropdown_symbol, div_dropdown_field], className='row'), ], className='custom-dropdown'),
    html.Div(html.Div(id='table-container_1'), style={'marginBottom': '15px', 'marginTop': '0px'}),
    html.Div(html.Div(id='table-container_2'), style={'marginBottom': '15px', 'marginTop': '0px'}),
  ], style=CONTENT_STYLE)


In [30]:
cols_colors_crossing_levels = ['movement']
df_crossing_levels[cols_colors_crossing_levels].quantile(0.1).items()

In [20]:
money                   = dash_table.FormatTemplate.money(2)
percentage              = dash_table.FormatTemplate.percentage(2)
def style_selected_rows(sel_rows):
    if sel_rows is None:                                                                                                                                                                                                                      
        return dash.no_update
    val = [
        {"if": {"filter_query": "{{id}} ={}".format(i)}, "backgroundColor": "#404040",}
        for i in sel_rows
    ]   
    return val

columns_crossing_levels = [
   dict(id='symbol',    name='Symbol'),
   dict(id='ago',       name='Time Ago'),
   dict(id='level',     name='Level',           type='numeric', format=percentage),
   dict(id='direction', name='Trend'),
   dict(id='crossing',  name='Type Crossing'),
   dict(id='dist',      name='Distance',        type='numeric', format=percentage),
   dict(id='movement',  name='Movement',        type='numeric', format=percentage)
]
cols_colors_crossing_levels = ['movement']

@app.callback(
    Output("table", "style_data_conditional"),
    Input("table", "derived_virtual_selected_row_ids"),
)

@app.callback([Output('table-container_1', 'children')],
              [Input(component_id='symbol-dropdown', component_property='value'),
               Input(component_id='field-dropdown', component_property='value')])
def display_value(symbol_chosen, field_chosen):
  return (dash_table.DataTable(columns=columns_crossing_levels,
                               id='datatable-crossing-levels',
                               style_data_conditional=(
                                    [{ 'if': { 'row_index': 'even', 'filter': 'row_index > num(2)' }, 'backgroundColor': '#EBEDEF' }] + 
                                    [{ 'if': { 'filter_query': '{{{}}} > 0'.format(col), 'column_id': col }, 'color': 'green'
                                     } for (col, value) in df_crossing_levels[cols_colors_crossing_levels].quantile(0.1).iteritems()] +
                                    [{ 'if': { 'filter_query': '{{{}}} < 0'.format(col), 'column_id': col }, 'color': 'tomato'
                                     } for (col, value) in df_crossing_levels[cols_colors_crossing_levels].iteritems()] 
                                ),
                               data=df_crossing_levels.sort_values(by=[field_chosen]).to_dict('records'),
                               export_format="csv",
                               style_as_list_view=True,
                               fill_width=True,
                               sort_action='native',
                               sort_mode='single',
                               style_cell={'font-size': '12px'},
                               style_header={'backgroundColor': 'black', 'color': 'white', },
                               fixed_rows={'headers': True, 'data': 0},
                               ),)

columns_positions       = [
    dict(id='symbol', name='Symbol'),
    dict(id='amount', name='Amount', type='numeric', format=money),
    
    dict(id='quote_purchase', name='Purchase ($)', type='numeric', format=money),
    dict(id='quote_last_quote', name='Last Quote ($)', type='numeric', format=money),
    dict(id='ticks_yesterday', name='Yesterday ($)', type='numeric', format=money),
    
    dict(id='profit_historical', name='Profit ($)', type='numeric', format=money),
    dict(id='profits_last_day', name='Last Day ($)', type='numeric', format=money),
    
    dict(id='profit_historical_perc', name='Profit (%)', type='numeric', format=percentage),
    dict(id='profits_last_day_perc', name='Last Day (%)', type='numeric', format=percentage),
    dict(id='participation', name='Weight', type='numeric', format=percentage)
]
cols_colors_positions = ['profit_historical','profit_historical_perc','profits_last_day','profits_last_day_perc']


@app.callback(
    Output('table-container_2', 'children'),
    [Input(component_id='symbol-dropdown', component_property='value'),
     Input(component_id='field-dropdown', component_property='value')]
)
def display_value(symbol_chosen, field_chosen):
  return (dash_table.DataTable(columns=columns_positions,
                               id='datatable-positions',
                               style_data_conditional=(
                                    [
                                        { 'if': { 'row_index': 'even', 'filter': 'row_index > num(2)' }, 'backgroundColor': '#EBEDEF' },
                                    ] + 
                                    [
                                        { 'if': { 'filter_query': '{{{}}} > 0'.format(col, value), 'column_id': col }, 'color': 'green'
                                        } for (col, value) in df_positions[cols_colors_positions].quantile(0.1).iteritems()
                                    ] +
                                    [
                                        { 'if': { 'filter_query': '{{{}}} < 0'.format(col, value), 'column_id': col }, 'color': 'tomato'
                                        } for (col, value) in df_positions[cols_colors_positions].iteritems()
                                    ] 
                                ),
                               data=df_positions.sort_values(by=[field_chosen]).to_dict('records'),
                               export_format="csv",
                               style_as_list_view=True,
                               fill_width=True,
                               sort_action='native',
                               sort_mode='single',
                               style_cell={'font-size': '12px'},
                               style_header={'backgroundColor': 'black', 'color': 'white', },
                               fixed_rows={'headers': True, 'data': 0},
                               ),)

In [21]:
#####################################################
# Part 6: Set up local server to show the dashboard
#####################################################
if __name__ == '__main__':
    app.run_server(debug=False, port=8008)
    #   app.run(debug=True)

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115.py:37: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115.py:38: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115.py:84: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115.py:86: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115.py:37: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

/var/folders/yy/t103484j0x128qwsqnsn13zm0000gp/T/ipykernel_31098/2766165115